In [ ]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv(override=True)

# Create Ollama Chat Model
from langchain_ollama import ChatOllama
llama_chat = ChatOllama(base_url="http://localhost:11434",
                        model="qwen3:8b",
                        temperature=0,
                        max_tokens=100)


[Document(id='71395a01-7438-4280-8c43-5c538efbc2f1', metadata={}, page_content='Cypress has built-in support for retries, time-travel debugging, and automatic waiting, making it easy to test dynamic web pages.'),
 Document(id='62742469-b2da-46b5-a01e-cec606bef560', metadata={}, page_content='Cypress has built-in support for retries, time-travel debugging, and automatic waiting, making it easy to test dynamic web pages.'),
 Document(id='667898a4-9134-4095-8936-7ea74effb4f6', metadata={}, page_content='Playwright allows network interception, headless execution, and tracing for debugging complex web applications.')]

#### Define sample Documents

In [ ]:
# Define some Documents
from langchain_core.documents import Document
docs = [
    Document(page_content="Playwright is a modern automation library for end-to-end testing. It supports multiple browsers like Chromium, Firefox, and WebKit."),
    Document(page_content="Selenium is a widely used open-source framework for web automation, supporting multiple programming languages and browsers."),
    Document(page_content="Cypress is a JavaScript-based testing tool primarily used for front-end testing. It runs in the browser and provides fast feedback loops."),
    Document(page_content="Playwright allows network interception, headless execution, and tracing for debugging complex web applications."),
    Document(page_content="Selenium WebDriver enables automated browser testing using various bindings such as Python, Java, and C#."),
    Document(page_content="Cypress has built-in support for retries, time-travel debugging, and automatic waiting, making it easy to test dynamic web pages."),
    Document(page_content="This document talks about REST API testing tools, which are unrelated to Playwright, Selenium, or Cypress.")
]

questions = [
    "What is Playwright and what browsers does it support?",
    "What is Selenium and what programming languages does it support?",
    "What is Cypress, and how is it different from Selenium?",
    "How does Playwright handle network interception and debugging?",
    "What are the key features of Selenium WebDriver?",
    "What are the main advantages of Cypress for testing?",
    "What are REST API testing tools, and how are they different from browser automation tools?"
]

#### Create Vector Store and Retriever

In [14]:

# Create Ollama Embeddings
from langchain_ollama import OllamaEmbeddings
ollama_embeddings = OllamaEmbeddings(model="llama3.2:latest")

# Create Vector Storage
from langchain_chroma import Chroma
vector_store = Chroma.from_documents(documents=docs, 
                                     embedding=ollama_embeddings)
# Create Retriever
retriever = vector_store.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k": 3}
)
# Retrieve Documents
retrieved_docs = retriever.invoke("What is playwright")
retrieved_docs

[Document(id='62742469-b2da-46b5-a01e-cec606bef560', metadata={}, page_content='Cypress has built-in support for retries, time-travel debugging, and automatic waiting, making it easy to test dynamic web pages.'),
 Document(id='0b2117ce-14db-4930-939c-0c2e4d0b14ef', metadata={}, page_content='Cypress has built-in support for retries, time-travel debugging, and automatic waiting, making it easy to test dynamic web pages.'),
 Document(id='87988330-0511-4f66-966e-4433fdcc7afb', metadata={}, page_content='Cypress has built-in support for retries, time-travel debugging, and automatic waiting, making it easy to test dynamic web pages.')]

#### Create QA RAG Chain

In [11]:
### Create a QA Chain to invoke
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
system_prompt = """
"You are an assistance for question-answering task"
    "Use the following pieces of retrieved context to answer the question."
    "If you don't know the answer, just say that you don't know, don't try to make up an answer."
    "Use three sentences maximum to answer the question and keep the answer concise."
    "The answer should be in bullet points."
    "\n\n{context}\n\n"
"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{question}")
])

def format_doc(docs):
    return "\n\n".join([doc.page_content for doc in docs])

rag_chain = (
    {
        "context": retriever | format_doc,
        "question": RunnablePassthrough()
    } 
    | prompt_template 
    | llama_chat
    | StrOutputParser()
)

In [13]:
response = rag_chain.invoke("What is PlayWright")
print(response)

- Playwright is a tool for debugging complex web applications.  
- It enables network interception, headless execution, and tracing.  
- These features aid in testing and analyzing web behavior.


#### Creating Mutlishot Data Set for RAGAs

In [ ]:
dataset = []

for question, doc in zip(questions, docs):
    relevant_docs = [doc.page_content for doc in retriever.invoke(question)]
    response = rag_chain.invoke(question)
    dataset.append(
        {
            "user_input": question,
            "retrieved_contexts": relevant_docs,
            "response": response,
            "reference": doc.page_content
        }
    )
dataset

#### Evaluation Using RAGAs

In [ ]:
from ragas import EvaluationDataset
from ragas import evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

evaluation_dataset = EvaluationDataset.from_list(dataset)
result = evaluate(dataset= evaluation_dataset,
                  metrics=[LLMContextRecall(),
                           Faithfulness(), 
                           FactualCorrectness()],
                  llm= llama_chat)
result